In [1]:
from __future__ import annotations
from src.lib.utils.FileHandling.FileHandlingInterface import *
from src.lib.SVCDSeg.SVCD import *
import matplotlib.pyplot as plt
import logging
import time
import warnings

In [2]:
import numpy as np
import pandas as pd
import random

results = {}
from PIL import Image, ImageOps
# import cv2

In [3]:
def dice(true_image,generated_image):
    true_image = true_image.flatten()
    generated_image = generated_image.flatten()
    if len(true_image)!= len(generated_image):
        print('Error in input and output image')
    else:
        intersection = 0
        for ind in range(len(true_image)):
            if true_image[ind] == generated_image[ind]:
                intersection = intersection + 1
    return(2*intersection/ (2*len(true_image)))

In [4]:
fnames = ['2007_000175', '2007_000243','2007_000464','2007_001289','2007_001397','2007_001834']
lmbda = [8e-3, 8e-2, 8e-1]
#                 **** Add loops for added hyperparameters

for fname in fnames:
        for lm in lmbda:
            KEA_IMAGE_PATH = f"data/demoPascal/{fname}.jpg"
            KEA_SCRIBBLE_PATH = f"./data/demoPascal/{fname}.xml"

            sample_image_kea = TargetImage(
                KEA_IMAGE_PATH
            )
            sample_scribble_kea = EncodedScribble(
                KEA_SCRIBBLE_PATH,
            )
            n_classes = sample_scribble_kea.get_n_classes()
            segmenter = SVCDSeg(
                n_classes,
                max_iter = 1000,
                debug = 0,
                lambda_ = 8e-3,
                #tau_dual = 0.5 
            )
            fitted_segmentation = segmenter.fit(
                sample_image_kea,
                sample_scribble_kea
            )
            for class_theta in segmenter.theta_t1:
                final_segmentation = class_theta.transpose(1, 0)
            final_segmentation = np.argmax(segmenter.theta_t1, axis = 0).transpose(1, 0)
            np.savetxt(f'Dice_Score/Generated_maps/{fname}.csv', final_segmentation, delimiter=',', fmt='%.2f')


            generated_image = np.genfromtxt(f"C:/Users/arind/OneDrive/Desktop/Classes/GM_2/T2_DSBA_Graphical_Methods/Dice_Score/Generated_maps/{fname}.csv", delimiter=',')
            true_image = np.asarray(ImageOps.grayscale(Image.open(f"C:/Users/arind/OneDrive/Desktop/Classes/GM_2/T2_DSBA_Graphical_Methods/Dice_Score/Generated_maps/{fname}.png")))
            classes_true = np.unique(true_image.flatten())
            classes_predicted = np.unique(generated_image.flatten())

            delta = len(classes_predicted) - len(classes_true)
            if delta<0: delta = delta*-1
            if delta != 0:

                li = []
                while len(li)!= delta:
                    li.append(np.random.randint(300, 30000))

                if len(classes_predicted)>len(classes_true):
                    classes_true = np.append(classes_true,li)
                else:
                    classes_predicted = np.append(classes_predicted,li)



            final_mappings = []
            for i in  range (0, len(classes_predicted)):
                classes_predicted = np.concatenate((classes_predicted[-1:], classes_predicted[:-1]))
                li = []
                for mappings in np.column_stack((classes_true, classes_predicted)):
                    ct, cp = mappings[0], mappings[1] 
            #         print(ct, cp)
                    li.append((ct, cp))
            #     print(i,li)
                final_mappings.append(li)
            final_mappings
            dice_s = 0 
            for mapping in final_mappings:
                gi_list = list(generated_image.flatten())
                ti_list = list(true_image.flatten())
                for ct, cp in mapping:
                    rand_int = np.random.randint(300, 30000)
                    print(ct, '->' ,rand_int )
                    print(len(generated_image[generated_image == cp]))
                    gi = generated_image.copy()
                    gi[gi == cp] = rand_int
                    gi_list = [x if x!=cp else rand_int  for x in gi_list]     
                    ti_list = [x if x!=ct else rand_int  for x in ti_list]
                gi_image = np.array(gi_list).reshape(generated_image.shape)  
                ti_image = np.array(ti_list).reshape(true_image.shape)
                print('***', dice(ti_image,gi_image))
                if dice(ti_image,gi_image)>dice_s:
                    dice_s = dice(ti_image,gi_image)
#                 **** Change for added hyperparameters
                results[str(fname)+'|' + str(lm)] = dice_s

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]C:\Users\arind\OneDrive\Desktop\Classes\GM_2\T2_DSBA_Graphical_Methods\src\lib\probabilityEstim\Likelihood.py:168: RuntimeWarning: invalid value encountered in true_divide
  kde_likelihood_map /= np.sum(kde_likelihood_map, axis = 0) # normalize to sum to one over each class
  0%|                                                                                 | 1/1000 [00:02<34:06,  2.05s/it]

dtype float64


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:10<00:00, 14.09it/s]


0.0 -> 15273
0
76.0 -> 18562
47503
220.0 -> 29145
118497
*** 0.29212048192771084
0.0 -> 4900
118497
76.0 -> 1142
0
220.0 -> 5105
47503
*** 0.6334156626506025
0.0 -> 7123
47503
76.0 -> 18999
118497
220.0 -> 762
0
*** 0.07446385542168675


  0%|                                                                                 | 1/1000 [00:00<13:27,  1.24it/s]

...

0.0 -> 16085
0
133.0 -> 7786
32591
220.0 -> 11444
134409
*** 0.23403592814371257
0.0 -> 24180
134409
133.0 -> 26778
0
220.0 -> 21486
32591
*** 0.7435209580838323
0.0 -> 29617
32591
133.0 -> 25927
134409
220.0 -> 4601
0
*** 0.02244311377245509


  0%|▏                                                                                | 3/1000 [00:02<09:08,  1.82it/s]

dtype float64


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:09<00:00, 14.43it/s]


0.0 -> 12972
0
133.0 -> 9552
32591
220.0 -> 24059
134409
*** 0.23403592814371257
0.0 -> 19351
134409
133.0 -> 19934
0
220.0 -> 17182
32591
*** 0.7435209580838323
0.0 -> 28730
32591
133.0 -> 7965
134409
220.0 -> 5103
0
*** 0.02244311377245509


In [20]:
'a' + 'b'

'ab'

In [7]:

results
# df = pd.DataFrame(results)

{'2007_000175|0.008': 0.6334156626506025,
 '2007_000175|0.08': 0.6334156626506025,
 '2007_000175|0.8': 0.6334156626506025,
 '2007_000243|0.008': 0.9788888888888889,
 '2007_000243|0.08': 0.9788888888888889,
 '2007_000243|0.8': 0.9788888888888889,
 '2007_000464|0.008': 0.537968,
 '2007_000464|0.08': 0.537968,
 '2007_000464|0.8': 0.537968,
 '2007_001289|0.008': 0.9461941747572815,
 '2007_001289|0.08': 0.9461941747572815,
 '2007_001289|0.8': 0.9461941747572815,
 '2007_001397|0.008': 0.9515195195195195,
 '2007_001397|0.08': 0.9515195195195195,
 '2007_001397|0.8': 0.9515195195195195,
 '2007_001834|0.008': 0.7435209580838323,
 '2007_001834|0.08': 0.7435209580838323,
 '2007_001834|0.8': 0.7435209580838323}